## Model training on Argoverse Data

In [1]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms
import argparse

In [2]:
from utils.boxes import generate_anchors, xyxy_to_xywh, xywh_to_xyxy
from utils.boxes import compute_deltas, compute_overlaps, boxes_postprocess

In [3]:
from model.modules import deltas_to_boxes, compute_overlaps, safe_softmax
from utils.config import Config


In [4]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [5]:
args = Config().parse()

training chunk_sizes: [20]
The results will be saved to  C:\Users\cathx\repos\squeezedet\SqueezeDet-PyTorch\exp\default


In [6]:
from demo import demo

In [7]:
args.arch = 'squeezedet'
args.gpus = [0]
# args.device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
args.device = 'cuda:0'
args.root_dir = ''
args.batch_size = 15
args.num_epochs = 200
args.no_eval = True    # default = True, we do not want to surpass mAP evaluation during training
args.load_model = '../models/squeezenet1_1-f364aa15.pth'
# args.data_dir = 'C://users/cathx/repos/squeezedet/squeezedet-pytorch/data/data_kitti/'

In [8]:
from train import train

Aniyeo Tesla


In [10]:
# train(args)